In [2]:
import os, sys
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [3]:
# context manager to suppress stdoutm for annoying long printouts
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [4]:
experiment = 'vgg_imagenet'
batch_size = 300

In [5]:
platform = 'paperspace' #@param ['colab', 'paperspace'] {allow-input: true}

if platform == 'jupyter_notebook':
    challenge_data_dir = '../data/algonauts_2023_challenge_data'
    exp_output_dir = f'../data/out/{experiment}'

if platform == 'paperspace':
    challenge_data_dir = '/storage/algonauts_2023_challenge_data'
    exp_output_dir = f'/notebooks/data/out/{experiment}'

## Functions not aligned with the current structuree

In [ ]:
def train_pca(model, train_dataset):
    
    features = []
    for batch in train_dataset:
        # Extract features
        with HiddenPrints():
            ft = model.predict(batch)
        # Flatten the features
        ft = ft.reshape(ft.shape[0], -1)
        # Append to list
        features.append(ft)
    # Combine features from all batches
    features = np.vstack(features)
    n_components = 100
    print('Features shape: ', features.shape, ' PCA Components: ', n_components)
    # Fit PCA to combined features
    pca = PCA(n_components=n_components)
    pca.fit(features)
    # print('plot the explained variance')
    # plt.plot(pca.explained_variance_ratio_)
    # plt.show()
    # print('plot the cumulative explained variance')
    # plt.plot(np.cumsum(pca.explained_variance_ratio_))
    # plt.show()
    return pca

def extract_and_transform_features(dataset, model, pca):
    features = []
    for batch in dataset:
        with HiddenPrints():
            ft = model.predict(batch)
        # Flatten the features
        ft = ft.reshape(ft.shape[0], -1)
        # Fit PCA to batch of features
        ft = pca.transform(ft)
        features.append(ft)
    return np.vstack(features)

def load_datasets(dataset, transform_image):
    train_val_imgs_paths = [os.path.join(dataset.train_img_dir, img_name) for img_name in dataset.training_img_list]
    train_paths = [train_val_imgs_paths[i] for i in dataset.idxs_train]
    val_paths = [train_val_imgs_paths[i] for i in dataset.idxs_val]
    test_imgs_paths = [os.path.join(dataset.test_img_dir, img_name) for img_name in dataset.test_img_list]

    train_ds = tf.data.Dataset.from_tensor_slices(train_paths)
    val_ds = tf.data.Dataset.from_tensor_slices(val_paths)
    test_ds = tf.data.Dataset.from_tensor_slices(test_imgs_paths)

    def load_image(image_path):
        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        return img
    
    def preprocess(ds):
        ds = ds.map(load_image)
        ds = ds.map(transform_image)
        return ds
    
    train_ds = preprocess(train_ds).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = preprocess(val_ds).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_ds = preprocess(test_ds).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds, test_ds

def save_predictions(lh_fmri_test_pred, rh_fmri_test_pred, subject_submission_dir):
    lh_fmri_test_pred = lh_fmri_test_pred.astype(np.float32)
    rh_fmri_test_pred = rh_fmri_test_pred.astype(np.float32)

    np.save(os.path.join(subject_submission_dir, 'lh_pred_test.npy'), lh_fmri_test_pred)
    np.save(os.path.join(subject_submission_dir, 'rh_pred_test.npy'), rh_fmri_test_pred)

## Specify model and appropriate image preprocessing steps
Define a function to load and return the model, printing its layers.
Due to memory constraints, we will delete the model from memory in each loop. Here we first load it, print its
nodes and delete it.

### Choose to load a pretrained model from keras.applications

In [ ]:
def load_model(layer_name=None):
    model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)
    # Slice the model to extract features from a specific layer
    if layer_name is not None:
        model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    return model

def transform_image(image):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image

### Or choose to load a model from file

In [ ]:
model_filename = 'model.h5'

def load_model(layer_name=None):
    model = tf.keras.models.load_model(model_filename)
    # Slice the model to extract features from a specific layer
    if layer_name is not None:
        model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    return model

def transform_image(image):
    image = tf.image.resize(image, (227, 227))
    image = tf.cast(image, tf.float32) / 127.5 - 1.0
    mean = [0.485, 0.456, 0.406]
    variance = [0.229, 0.224, 0.225]
    normalization_layer = Normalization(mean=mean, variance=variance)
    image = normalization_layer(image)
    return image

### Print the model layers

In [ ]:
model = load_model()
print(*(layer.name for layer in model.layers), sep=' -> ')
del model

## Select layers and subjects
Now let's define which layer(s) we will pick from, and which subject(s) to process

In [ ]:
layers = ['block1_pool']
subjects = [
    1, 
    # 2, 3, 4, 5, 6, 7, 8
    ]


## Run pipeline

In [10]:
from src.algonauts.evaluators import correlations as corr
from src.algonauts.data_processors.nsd_dataset import NSDDataset

for layer_name in layers:
    print(f'Running for layer {layer_name}')
    for subj in subjects:
        print(f'Running for subject {subj}')

        # Set data directories based on parameters
        output_dir = f'{exp_output_dir}/{layer_name}'
        dataset = NSDDataset(challenge_data_dir, output_dir, subj)

        print('Loading datasets...')
        train_ds, val_ds, test_ds = load_datasets(dataset, transform_image)
        print('Datasets loaded')

        # Load model for the iteration
        model = load_model(layer_name)
        
        # Train PCA
        print('Training PCA...')
        pca = train_pca(model, train_ds)
        print('PCA over')

        # Extract and transform features
        print('Extracting and transforming features...')
        train_features = extract_and_transform_features(train_ds, model, pca)
        val_features = extract_and_transform_features(val_ds, model, pca)
        test_features = extract_and_transform_features(test_ds, model, pca)
        print('Features extracted and transformed')

        # Delete model to free up memory
        del model, pca

        # Fit regression
        print('Fitting regression...')
        reg_lh = LinearRegression().fit(train_features, dataset.lh_fmri_train)
        reg_rh = LinearRegression().fit(train_features, dataset.rh_fmri_train)
        print('Regression fitted')
        
        # Use fitted linear regressions to predict the validation and test fMRI data
        print('Predicting fMRI data...')
        lh_fmri_val_pred = reg_lh.predict(val_features)
        lh_fmri_test_pred = reg_lh.predict(test_features)
        rh_fmri_val_pred = reg_rh.predict(val_features)
        rh_fmri_test_pred = reg_rh.predict(test_features)
        print('fMRI data predicted')
        # Calculate correlations for each hemispher
        print('Calculating correlations...')
        lh_correlation = corr.calculate_correlation(lh_fmri_val_pred, dataset.lh_fmri_val)
        rh_correlation = corr.calculate_correlation(rh_fmri_val_pred, dataset.rh_fmri_val)  
        print('Correlations calculated')
    
        corr.plot_and_write_correlations(dataset, lh_correlation, rh_correlation, exp_output_dir, layer_name, subj)
        # Save test predictions
        np.save(os.path.join(dataset.subject_submission_dir, 'lh_pred_test.npy'), lh_fmri_test_pred)
        np.save(os.path.join(dataset.subject_submission_dir, 'rh_pred_test.npy'), rh_fmri_test_pred)

In [38]:
import json

def find_best_correlations(exp_output_dir, subject_id):
    json_path = f'{exp_output_dir}/results.json'
    with open(json_path, 'r') as f:
        data = json.load(f)

    layers_names_with_filtered_subjects = [
        (layer["layer_name"], subj)
        for layer in data["Layers"] 
        for subj in layer["Subjects"]
        if subj["subject"] == subject_id
    ]

    def max_layer_value(subj_data, hemi, roi):
        max_layer, max_value = max(
            (
                (layer_name, roi_corr.get(roi))
                for layer_name, subj in subj_data
                for roi_corr in [subj[f"{hemi}_median_roi_correlation"]]
                if roi_corr.get(roi) is not None
            ),
            key=lambda x: x[1],
            default=(None, None),
        )
        return max_layer, max_value

    roi_names = list(layers_names_with_filtered_subjects[0][1]["LH_median_roi_correlation"].keys())

    best_correlation = {
        hemi: {
            roi: {
                "layer": max_layer_value(layers_names_with_filtered_subjects, hemi, roi)[0],
                "value": max_layer_value(layers_names_with_filtered_subjects, hemi, roi)[1],
            }
            for roi in roi_names
        }
        for hemi in ["LH", "RH"]
    }

    return best_correlation


subj = 1
result = find_best_correlations(exp_output_dir, subj)
print(json.dumps(result, indent=2))


{
  "LH": {
    "V1v": {
      "layer": "block2_pool",
      "value": 0.5905791030486138
    },
    "V1d": {
      "layer": "block2_pool",
      "value": 0.5990486552463073
    },
    "V2v": {
      "layer": "block3_pool",
      "value": 0.5282678641036374
    },
    "V2d": {
      "layer": "block3_pool",
      "value": 0.5175909098720242
    },
    "V3v": {
      "layer": "block3_pool",
      "value": 0.5141076787726454
    },
    "V3d": {
      "layer": "block4_pool",
      "value": 0.5065629532466085
    },
    "hV4": {
      "layer": "block4_pool",
      "value": 0.44591214364383464
    },
    "EBA": {
      "layer": "block5_pool",
      "value": 0.4846002619845918
    },
    "FBA-1": {
      "layer": "block5_pool",
      "value": 0.4077181141971304
    },
    "FBA-2": {
      "layer": null,
      "value": null
    },
    "mTL-bodies": {
      "layer": null,
      "value": null
    },
    "OFA": {
      "layer": "block4_pool",
      "value": 0.43446271378361234
    },
    "FFA-1": 